In [ ]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import pymysql
import sys
import os

In [ ]:
# # OP.GG Database에 접근하기 위한 연결 객체 생성
# con = pymysql.connect(
#     user = os.environ['LOL_KR_ID'],
#     passwd = os.environ['LOL_KR_PW'],
#     host = os.environ['LOL_KR_HOST'],
#     db = 'lol',
#     charset = 'utf8'
# )
# cursor = con.cursor(pymysql.cursors.DictCursor)

In [ ]:
# # 데이터 구조 파악을 위한 sample data fetch
# # SQL
# cursor.execute('''
# SELECT STRAIGHT_JOIN *
# FROM opGame o FORCE INDEX (ix_createDate),
# p_opGameStats p FORCE INDEX (`PRIMARY`)
# WHERE o.gameId = p.gameId
# AND o.createDate >= '2021-08-12'
# AND p.createDate >= '2021-08-12'
# ORDER BY o.createDate DESC
# LIMIT 30
# ''')
# sample = cursor.fetchall()

# # DataFrame으로 변환
# sample = pd.DataFrame(sample)

In [ ]:
# sample.to_csv("sample.csv", mode='w')

In [ ]:
# 미리 뽑아둔 CSV 파일에서 가져오기
sample = pd.read_csv('data/sample.csv', index_col=0)

In [ ]:
# 데이터 확인
sample

In [ ]:
# Column 수가 많아서 print했을 때 다 보이게 옵션 설정
pd.set_option('display.max_columns', None)

# columns 확인
print(sample.columns)

# 값이 정상적인지 보기 위해 describe로 요약 정보 출력
sample.describe(include='all',datetime_is_numeric=True)

In [ ]:
# 필요한 값만 선택
sample = sample[['gameId','subType','gameLength','teamId','championId','result','position']]

In [ ]:
# 최종적으로 만들어진 sample dataframe
sample

In [ ]:
# Reset sample data
%reset

In [1]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import pymysql
import sys
import os
from scipy.stats import norm
import requests

In [ ]:
# # OP.GG Database에 접근하기 위한 연결 객체 생성
# con = pymysql.connect(
#     user = os.environ['LOL_KR_ID'],
#     passwd = os.environ['LOL_KR_PW'],
#     host = os.environ['LOL_KR_HOST'],
#     db = 'lol',
#     charset = 'utf8'
# )
# cursor = con.cursor(pymysql.cursors.DictCursor)

In [ ]:
# # Full data fetch
# # SQL
# cursor.execute('''
# SELECT STRAIGHT_JOIN o.gameId, subType, gameLength, teamId, championId, result, position
# FROM opGame o FORCE INDEX (ix_createDate),
# p_opGameStats p FORCE INDEX (`PRIMARY`)
# WHERE o.gameId = p.gameId
# AND o.createDate >= '2021-08-12'
# AND p.createDate >= '2021-08-12'
# LIMIT 1000000
# ''')
# gamestats = cursor.fetchall()

# # DataFrame으로 변환
# gamestats = pd.DataFrame(gamestats)

In [ ]:
# gamestats.to_csv("gamestats.csv", mode='w')

In [2]:
# 미리 뽑아둔 CSV 파일에서 가져오기
gamestats = pd.read_csv('data/gamestats.csv', index_col=0)

In [3]:
# 데이터 확인
gamestats

,gameId,subType,gameLength,teamId,championId,result,position
0,5385587730,440,1804,200,81,WIN,A
1,5385587730,440,1804,200,107,WIN,J
2,5385587730,440,1804,100,121,LOSE,J
3,5385587730,440,1804,100,517,LOSE,M
4,5385587730,440,1804,100,12,LOSE,S
...,...,...,...,...,...,...,...
99995,5385598894,440,1977,200,222,WIN,A
99996,5385598894,440,1977,200,234,WIN,J
99997,5385598894,440,1977,100,81,LOSE,A
99998,5385598894,440,1977,100,1,LOSE,M


In [4]:
# 데이터 타입 확인
print(gamestats.dtypes)

# 데이터 요약 정보
gamestats.describe(include='all',datetime_is_numeric=True)

gameId         int64
subType        int64
gameLength     int64
teamId         int64
championId     int64
result        object
position      object
dtype: object


,gameId,subType,gameLength,teamId,championId,result,position
count,1.000000e+05,100000.000000,100000.000000,100000.000000,100000.000000,100000,66998
unique,NaN,NaN,NaN,NaN,NaN,3,5
top,NaN,NaN,NaN,NaN,NaN,WIN,A
freq,NaN,NaN,NaN,NaN,NaN,49677,13400
mean,5.385674e+09,447.026200,1491.257480,149.990000,149.647020,NaN,NaN
std,5.216472e+04,111.647298,489.932212,50.000249,178.131795,NaN,NaN
min,5.385581e+09,420.000000,190.000000,100.000000,1.000000,NaN,NaN
25%,5.385629e+09,420.000000,1141.000000,100.000000,43.000000,NaN,NaN
50%,5.385673e+09,430.000000,1440.000000,100.000000,86.000000,NaN,NaN
75%,5.385712e+09,450.000000,1826.000000,200.000000,164.000000,NaN,NaN


In [5]:
# 각 column 별로 결측치 확인
gamestats.isnull().sum()

gameId            0
subType           0
gameLength        0
teamId            0
championId        0
result            0
position      33002
dtype: int64

In [6]:
# 'result' column에 있는 값 확인
gamestats['result'].unique()

array(['WIN', 'LOSE', 'UNKNOWN'], dtype=object)

In [7]:
# 'UNKNOWN'인 case 확인
gamestats[gamestats['result'] == 'UNKNOWN']

,gameId,subType,gameLength,teamId,championId,result,position
300,5385762506,430,196,200,134,UNKNOWN,M
301,5385762506,430,196,200,51,UNKNOWN,T
302,5385762506,430,196,200,23,UNKNOWN,A
303,5385762506,430,196,100,157,UNKNOWN,M
304,5385762506,430,196,200,12,UNKNOWN,S
...,...,...,...,...,...,...,...
97396,5385773777,440,194,200,51,UNKNOWN,A
97397,5385773777,440,194,200,238,UNKNOWN,M
97398,5385773777,440,194,100,16,UNKNOWN,S
97399,5385773777,440,194,200,59,UNKNOWN,J


In [8]:
# 'UNKNOWN'인 case drop
gamestats = gamestats[gamestats['result'] != 'UNKNOWN']

In [9]:
# 'subType' 확인
gamestats['subType'].value_counts()

420     35380
450     30850
430     20030
440     10928
850       780
830       710
840       280
2020      230
2010      120
2000       22
Name: subType, dtype: int64

In [10]:
# 솔랭(420), 칼바람(450)만 남기기
gamestats = gamestats[gamestats['subType'].isin([420,450])]

In [11]:
# 데이터 타입 categorical로 변환
gamestats['teamId'] = pd.Categorical(gamestats.teamId)
gamestats['subType'] = pd.Categorical(gamestats.subType)
gamestats['position'] = pd.Categorical(gamestats.position)

In [12]:
# 솔랭 게임 중 포지션 분류 안 된 게임 확인
gamestats['position'][gamestats['subType'] == 420].isnull().sum()

0

In [13]:
# 포지션 분류 안 된 게임 drop
gamestats = gamestats.drop(gamestats[(gamestats['subType']==420) & (gamestats['position'].isnull())].index)

In [14]:
# 'WIN', 'LOSE' -> True, False로 다루기 용이하게 변환
gamestats['result'] = (gamestats['result'] == 'WIN')

In [15]:
# 전처리된 DataFrame에서 row case 재확인
gamestats[['teamId','subType','result']].drop_duplicates().sort_values(["subType",'result','teamId'])

,teamId,subType,result
180,100,420,False
33,200,420,False
30,100,420,True
182,200,420,True
52,100,450,False
10,200,450,False
11,100,450,True
50,200,450,True


In [16]:
# subType, championId 별로 블루팀 승률, 레드팀 승률, 각 진영 게임 수 확인
def func_calc_WR(x):
        d = []
        d.append(sum(x['teamId']==100)),
        d.append(x['result'][x['teamId']==100].sum() / sum(x['teamId']==100)),
        d.append(sum(x['teamId']==200)),
        d.append(x['result'][x['teamId']==200].sum() / sum(x['teamId']==200)),
        
        return pd.Series(d, index=['BLUE_count', 'BLUE_WR', 'RED_count', 'RED_WR'])

summary = gamestats.groupby(['subType','championId'], as_index=False).apply(func_calc_WR)

In [17]:
# Summary 확인
summary

,subType,championId,BLUE_count,BLUE_WR,RED_count,RED_WR
0,420,1,34.0,0.588235,34.0,0.588235
1,420,2,20.0,0.700000,28.0,0.464286
2,420,3,91.0,0.538462,92.0,0.489130
3,420,4,66.0,0.424242,69.0,0.449275
4,420,5,154.0,0.474026,169.0,0.449704
...,...,...,...,...,...,...
307,450,555,208.0,0.480769,200.0,0.455000
308,450,777,92.0,0.500000,89.0,0.426966
309,450,875,101.0,0.623762,120.0,0.450000
310,450,876,71.0,0.591549,68.0,0.411765


In [18]:
# 유의수준 0.005에서 검증
summary['p_value'] = (2*norm.cdf(-abs(summary.RED_WR - summary.BLUE_WR)/
                                 np.sqrt(summary.BLUE_WR*(1-summary.BLUE_WR)/summary.BLUE_count+
                                         summary.RED_WR*(1-summary.RED_WR)/summary.RED_count)))
summary['unbalanced'] = summary['p_value'] < 0.005

In [19]:
summary[summary['unbalanced']].sort_values('p_value')

,subType,championId,BLUE_count,BLUE_WR,RED_count,RED_WR,p_value,unbalanced
180,450,25,192.0,0.682292,199.0,0.462312,0.000006,True
169,450,14,43.0,0.744186,42.0,0.380952,0.000289,True
195,450,40,135.0,0.629630,114.0,0.412281,0.000462,True
131,420,246,49.0,0.244898,57.0,0.526316,0.001823,True
244,450,105,101.0,0.544554,96.0,0.343750,0.003771,True


In [20]:
# 숫자로 된 각종 Key값에 대한 Value 가져오기
champion_constant = requests.get("http://ddragon.leagueoflegends.com/cdn/11.16.1/data/ko_KR/champion.json")
# Json 파일을 DataFrame으로 변환
champion_df = pd.DataFrame(champion_constant.json()['data']).T[['key','name']]
# 변수형 문자 -> 숫자
champion_df['key'] = pd.to_numeric(champion_df['key'])

In [21]:
# 매칭되는 곳으로 내용 추가하여 새로운 DataFrame으로 저장
summary = pd.merge(summary,champion_df,how = 'left', left_on = 'championId', right_on = 'key')
# 중복되는 값 삭제
summary = summary.rename(columns={'name': 'Champion'}).drop(['championId','key'],axis = 1)
summary_View = summary[['subType','Champion','BLUE_WR','RED_WR','unbalanced','p_value']]

In [22]:
summary_View[summary_View['unbalanced']].sort_values('p_value')

,subType,Champion,BLUE_WR,RED_WR,unbalanced,p_value
180,450,모르가나,0.682292,0.462312,True,0.000006
169,450,사이온,0.744186,0.380952,True,0.000289
195,450,잔나,0.629630,0.412281,True,0.000462
131,420,키아나,0.244898,0.526316,True,0.001823
244,450,피즈,0.544554,0.343750,True,0.003771


In [23]:
summary2 =summary.sort_values('p_value')
summary2['unbalanced'].value_counts()

False    307
True       5
Name: unbalanced, dtype: int64

In [24]:
summary2['subType'].value_counts()

420    156
450    156
Name: subType, dtype: int64

In [25]:
summary2[summary2['subType']==420][['BLUE_WR','RED_WR']].mean()

BLUE_WR    0.505397
RED_WR     0.506826
dtype: float64

In [26]:
summary2[summary2['subType']==420].sort_values(by='BLUE_WR',ascending=0 )

,subType,BLUE_count,BLUE_WR,RED_count,RED_WR,p_value,unbalanced,Champion
63,420,9.0,0.777778,9.0,0.555556,0.303484,False,스카너
107,420,14.0,0.714286,12.0,0.416667,0.110785,False,아우렐리온 솔
1,420,20.0,0.700000,28.0,0.464286,0.090442,False,올라프
94,420,24.0,0.666667,30.0,0.633333,0.798217,False,세주아니
140,420,38.0,0.657895,33.0,0.424242,0.042955,False,렉사이
...,...,...,...,...,...,...,...,...
145,420,35.0,0.371429,43.0,0.465116,0.401222,False,자야
117,420,19.0,0.368421,14.0,0.428571,0.727242,False,탈리야
105,420,28.0,0.321429,39.0,0.589744,0.023317,False,퀸
67,420,26.0,0.269231,17.0,0.470588,0.176776,False,우디르


## 내가 해보는거

In [27]:
gamestats2 = pd.read_csv('data/gamestats.csv', index_col=0)

In [29]:
gamestats2 = gamestats2[gamestats2['subType'].isin([450])]